In [1]:
%load_ext autoreload 
%autoreload 2
import newspaper
import rag

/opt/homebrew/Caskroom/miniforge/base/envs/newsrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
guardian = newspaper.build("http://theguardian.com/uk")

In [18]:
for article in guardian.articles[:10]:
    print(article.url)
    article.download()
    article.parse()
    print(article.text[:100])

https://www.theguardian.com/us-news/ng-interactive/2024/nov/06/us-election-results-map-2024-live-donald-trump-kamala-harris-president
Kamala Harris 226 Electoral college votes Donald Trump 295 Electoral college votes 67,978,219 votes 
https://www.theguardian.com/us-news/ng-interactive/2024/nov/06/us-election-results-map-2024-live-donald-trump-kamala-harris-president#how-does-the-us-election-work
Kamala Harris 226 Electoral college votes Donald Trump 295 Electoral college votes 67,978,219 votes 
http://theguardian.com/us-news/live/2024/nov/07/us-election-2024-live-updates-donald-trump-victory-kamala-harris
From 1h ago 13.50 GMT Biden to address nation after Trump re-elected president Joe Biden is schedule
http://theguardian.com/us-news/2024/nov/06/kamala-harris-concession-speech
Kamala Harris formally conceded the election to Donald Trump on Wednesday, urging Americans devastat
http://theguardian.com/us-news/2024/nov/07/election-trump-antivax-movement
Donald Trump will be propelled to t

In [12]:
len(guardian.articles)

1585

In [20]:
article = guardian.articles[0]

In [24]:
article.download()

In [27]:
article.parse()

In [29]:
article.title

'Putin congratulates Trump on election win and says Russia ready for dialogue'

In [36]:
len(d["entries"])

125

In [53]:
from rag import RAG, get_news

In [54]:
import feedparser
d = feedparser.parse("https://theguardian.com/uk/rss")


In [55]:
news = get_news("https://theguardian.com/uk/rss")
rag = RAG(documents=news)



In [56]:
results = rag.run("How are the Democrats taking the result of the US election?")

In [49]:
results.keys()

dict_keys(['llm', 'retriever', 'prompt_builder'])

In [50]:
print(results["prompt_builder"]["prompt"])


    Given these news article summaries, answer the question.
    Documents:
    
        <p>Preliminary exit polling shows Harris won women, but in smaller numbers than her Democratic predecessors</p><ul><li><a href="https://www.theguardian.com/us-news/ng-interactive/2024/nov/06/us-election-results-map-2024-live-donald-trump-kamala-harris-president">US election results 2024: live map and tracker</a></li></ul><p>Gender had promised to be one of<strong> </strong>the biggest stories of the 2024 election. With abortion one of voters’ top issues,<strong> </strong><a href="https://www.theguardian.com/us-news/donaldtrump">Donald Trump</a>’s well-documented history of misogyny<strong> </strong>and a female presidential candidate of color on the ballot, <a href="https://www.theguardian.com/us-news/democrats">Democrats</a> banked on women showing up in force to defeat him. But Trump’s stunning victory in both the electoral college and the popular vote dashed those hopes – and scrambled the narr

In [51]:
print(results["llm"]["replies"])

['The text does not provide information on how the Democrats are taking the result of the US election. However, it mentions that there is fear rippling through undocumented communities across the US as Donald Trump prepares to take the White House, promising record deportations, and that many men could not bring themselves to vote for a woman, with one analyst describing the racial and gender dimension of this as "bitter".']


In [58]:
from haystack import Document
d = Document(content="kk")

In [59]:
d.content


'kk'